In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from functions import *

Players

In [2]:
soup = get_soup('https://www.basketball-reference.com/players/a/antetgi01.html')

In [84]:

### Individual Per Game data: Regular Season and Playoff

def player_avg_data(player_username):

    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    per_game_data = soup.select('table[data-soc-sum-table-type="PlayerPerGame"]')

    if not per_game_data:
        print('No Data Found')
        exit

    ### Regular Season Table

    per_game_reg_season_columns = per_game_data[0].select('th[scope="col"]')
    per_game_reg_season_columns = [
        col.get_text(strip=True) for col in per_game_reg_season_columns
    ]

    per_game_reg_season_table_body = per_game_data[0].select('tbody')[0].select('th,td')
    table_data = []
    for i in per_game_reg_season_table_body:
        table_data.append(i.get_text(strip=True))

    reg_season_data = []
    BATCH_SIZE = 31
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        reg_season_data.append(curr)

    reg_season = pd.DataFrame(reg_season_data)
    reg_season.columns = per_game_reg_season_columns

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoff Table

    try:
        per_game_playoff_columns = per_game_data[1].select('th[scope="col"]')
    except IndexError as e:
        print('No Data Found')

    per_game_playoff_columns = [
        col.get_text(strip=True) for col in per_game_playoff_columns
    ]

    per_game_playoff_table_body = per_game_data[1].select('tbody')[0].select('th,td')
    table_data = []
    for i in per_game_playoff_table_body:
        table_data.append(i.get_text(strip=True))

    playoff_data = []
    BATCH_SIZE = 31
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        playoff_data.append(curr)

    playoff_season = pd.DataFrame(playoff_data)
    playoff_season.columns = per_game_playoff_columns

    playoff_season = playoff_season[
        playoff_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    return reg_season, playoff_season


In [85]:
def player_sum_data(player_username):
    
    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    total_data = soup.select('table[data-soc-sum-table-type="PlayerTotals"]')

    if not total_data:
        print('No Data Found')
        # exit

    ### Regular Season

    reg_data = total_data[0]
    reg_cols = [col.get_text() for col in reg_data.select('th[scope="col"]')]

    total_reg_season_table_body = reg_data.select('tbody')[0].select('th,td')
    table_data = []
    for i in total_reg_season_table_body:
        table_data.append(i.get_text(strip=True))

    reg_season_data = []
    BATCH_SIZE = len(reg_cols)
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        reg_season_data.append(curr)

    reg_season = pd.DataFrame(reg_season_data)
    reg_season.columns = reg_cols

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoffs

    try:
        post_data = total_data[1]
    except IndexError as e:
        print('No Data Found')
        
    post_cols = [col.get_text() for col in post_data.select('th[scope="col"]')]

    total_post_season_table_body = post_data.select('tbody')[0].select('th,td')
    table_data = []
    for i in total_post_season_table_body:
        table_data.append(i.get_text(strip=True))

    post_season_data = []
    BATCH_SIZE = len(post_cols)
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        post_season_data.append(curr)

    post_season = pd.DataFrame(post_season_data)
    post_season.columns = post_cols

    post_season = post_season[
        post_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    return reg_season, post_season

In [86]:
player_avg_data('antetgi01')[0]

,Season,Age,Team,Lg,Pos,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards
0,2013-14,19,MIL,NBA,SF,77,23,24.6,2.2,5.4,...,1.0,3.4,4.4,1.9,0.8,0.8,1.6,2.2,6.8,ROY-7
1,2014-15,20,MIL,NBA,SG,81,71,31.4,4.7,9.6,...,1.2,5.5,6.7,2.6,0.9,1.0,2.1,3.1,12.7,
2,2015-16,21,MIL,NBA,PG,80,79,35.3,6.4,12.7,...,1.4,6.2,7.7,4.3,1.2,1.4,2.6,3.2,16.9,
3,2016-17,22,MIL,NBA,SF,80,80,35.6,8.2,15.7,...,1.8,7.0,8.8,5.4,1.6,1.9,2.9,3.1,22.9,"MVP-7,MIP-1,AS,NBA2,DEF2"
4,2017-18,23,MIL,NBA,PF,75,75,36.7,9.9,18.7,...,2.1,8.0,10.0,4.8,1.5,1.4,3.0,3.1,26.9,"MVP-6,AS,NBA2"
5,2018-19,24,MIL,NBA,PF,72,72,32.8,10.0,17.3,...,2.2,10.3,12.5,5.9,1.3,1.5,3.7,3.2,27.7,"MVP-1,DPOY-2,AS,NBA1,DEF1"
6,2019-20,25,MIL,NBA,PF,63,63,30.4,10.9,19.7,...,2.2,11.4,13.6,5.6,1.0,1.0,3.7,3.1,29.5,"MVP-1,DPOY-1,AS,NBA1,DEF1"
7,2020-21,26,MIL,NBA,PF,61,61,33.0,10.3,18.0,...,1.6,9.4,11.0,5.9,1.2,1.2,3.4,2.8,28.1,"MVP-4,DPOY-5,AS,NBA1,DEF1"
8,2021-22,27,MIL,NBA,PF,67,67,32.9,10.3,18.6,...,2.0,9.6,11.6,5.8,1.1,1.4,3.3,3.2,29.9,"MVP-3,DPOY-6,AS,NBA1,DEF1"
9,2022-23,28,MIL,NBA,PF,63,63,32.1,11.2,20.3,...,2.2,9.6,11.8,5.7,0.8,0.8,3.9,3.1,31.1,"MVP-3,DPOY-6,AS,NBA1"


In [87]:
player_sum_data('antetgi01')[0]

,Season,Age,Team,Lg,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Trp-Dbl,Awards
0,2013-14,19,MIL,NBA,SF,77,23,1897,173,418,...,261,339,150,60,61,122,173,525,0,ROY-7
1,2014-15,20,MIL,NBA,SG,81,71,2541,383,780,...,442,542,207,73,85,173,254,1030,0,
2,2015-16,21,MIL,NBA,PG,80,79,2823,513,1013,...,499,612,345,94,113,208,258,1350,5,
3,2016-17,22,MIL,NBA,SF,80,80,2845,656,1259,...,558,700,434,131,151,234,246,1832,3,"MVP-7,MIP-1,AS,NBA2,DEF2"
4,2017-18,23,MIL,NBA,PF,75,75,2756,742,1402,...,597,753,361,109,106,223,231,2014,1,"MVP-6,AS,NBA2"
5,2018-19,24,MIL,NBA,PF,72,72,2358,721,1247,...,739,898,424,92,110,268,232,1994,5,"MVP-1,DPOY-2,AS,NBA1,DEF1"
6,2019-20,25,MIL,NBA,PF,63,63,1917,685,1238,...,716,856,354,61,66,230,195,1857,4,"MVP-1,DPOY-1,AS,NBA1,DEF1"
7,2020-21,26,MIL,NBA,PF,61,61,2013,626,1100,...,574,671,357,72,73,207,168,1717,7,"MVP-4,DPOY-5,AS,NBA1,DEF1"
8,2021-22,27,MIL,NBA,PF,67,67,2204,689,1245,...,644,778,388,72,91,219,212,2002,4,"MVP-3,DPOY-6,AS,NBA1,DEF1"
9,2022-23,28,MIL,NBA,PF,63,63,2024,707,1278,...,605,742,359,52,51,246,197,1959,6,"MVP-3,DPOY-6,AS,NBA1"
